In [1]:
from getpass import getuser # Libaray to copy things
from pathlib import Path # Object oriented libary to deal with paths
import os
from tempfile import NamedTemporaryFile, TemporaryDirectory # Creating temporary Files/Dirs
from subprocess import run, PIPE
import sys
 
import dask # Distributed data libary
from dask_jobqueue import SLURMCluster # Setting up distributed memories via slurm
from distributed import Client, progress, wait # Libaray to orchestrate distributed resources

import xarray as xr # Libary to work with labeled n-dimensional data and dask
import numpy as np
import matplotlib.pyplot as plt
# import metpy.calc as mpcalc

In [2]:
import warnings
warnings.filterwarnings(action='ignore')

In [4]:
dask_client = Client()
dask_client

Client Scheduler: tcp://127.0.0.1:37464 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 6 Cores: 36 Memory: 524.31 GB


In [5]:
data_path = Path('/work/mh0731/m300414/DyWinter_b10/Cartesian_Grid/')

glob_pattern_2d = 'div900_*.nc' 
div_files = sorted([str(f) for f in data_path.rglob(f'{glob_pattern_2d}')])[-31:]
glob_pattern_2d = '*mask.nc'
mask_files = sorted([str(f) for f in data_path.rglob(f'{glob_pattern_2d}')])

In [6]:
div = xr.open_mfdataset(div_files)['div'].squeeze()
mask = xr.open_mfdataset(mask_files)['high_rome_mask'].squeeze()

In [7]:
div

,Array,Chunk
Bytes,47.99 GB,1.59 GB
Shape,"(211, 1777, 16000)","(7, 1777, 16000)"
Count,93 Tasks,31 Chunks
Type,float64,numpy.ndarray


In [8]:
mask

,Array,Chunk
Bytes,6.00 GB,199.02 MB
Shape,"(211, 1777, 16000)","(7, 1777, 16000)"
Count,93 Tasks,31 Chunks
Type,bool,numpy.ndarray


In [9]:
# div_mean = div.mean(dim='time')
# div_mean_romep90 = div.where(mask, other=np.nan).mean(dim='time')
# div_mean_romep90 = div.where(mask, other=np.nan).sum(dim='time') / len(div['time'])

In [34]:
div_day = div[:-1].groupby('time.time').mean()

In [31]:
import pandas as pd
tt = pd.to_datetime(div.time.values)

In [33]:
tt.hour == 0

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,

In [37]:
div_day['time'] = div[:7]['time']

In [38]:
div_day.to_netcdf('/work/mh0731/m300414/DyWinter_b10/Cartesian_Grid/div_dailycycle.nc')